In [5]:
%load_ext autoreload
%autoreload 1

import json
import pandas
import os
import ipywidgets as ipyw
import matplotlib.pyplot as plt

from IPython.display import Image, display
from pprint import pprint
from ipywidgets import fixed, interactive_output
%aimport utils
from utils import *

pandas.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Ideas

  - If there are too many really dark pixels, the image is underexposed and discard it. Two ways to do this:
  
    1) Transform to HSV space, perform binary threshold on the Value channel. 
    2) Transform to greyscale, perform binary threshold on greyscale image
    
    In either case, if number of resulting black pixels is too large, image is underexposed
  - Same as above but with light pixels. So "inverted" binary threshold (or something like that)

In [16]:
run_idealo_iqa("technical", rebuild=True, gpu=False)

Sending build context to Docker daemon  4.637GB
Step 1/8 : FROM tensorflow/tensorflow:2.0.0-py3
 ---> 90f5cb97b18f
Step 2/8 : RUN apt-get update && apt-get install -y --no-install-recommends       bzip2       g++       git       graphviz       libgl1-mesa-glx       libhdf5-dev       openmpi-bin       wget &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 4e11f11efbce
Step 3/8 : COPY src /src
 ---> 193501443da7
Step 4/8 : COPY entrypoints /src/entrypoints
 ---> d8b6dacede56
Step 5/8 : WORKDIR /src
 ---> Running in 89e2649a031d
Removing intermediate container 89e2649a031d
 ---> 6f863a94edbc
Step 6/8 : RUN pip install -r requirements.txt
 ---> Running in 0e2563e248ab
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=40f2bcbae3cf5f1f23bf32a08e50475b1da789aa245a7149a44f8e5ba9471e6d
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You should consider upgrading v

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
scores = load_data("output.txt")
df = pandas.DataFrame(scores)
compute_image_statistics(df, base="./src/tests/test_images/")
has_yellow = df["image_id"].str.contains("yellow") 
is_off = df["color"]=="off"
brightness_eq_1 = df["led-brightness"]==1
brightness_eq_75 = df["led-brightness"]==0.75
exposure2000 = df["exposure"]==2000
exposure1000 = df["exposure"]==1000
full_strip = df["orientation"]=="full-strip"
df

# Technical Model

In [ ]:
ui, out = build_widgets(df, plot=False)

In [ ]:
e)
display(ui, out)

In [ ]:
df.describe()

In [ ]:
df.nlargest(10, "score")

In [ ]:
display_images_matching_criteria(df, count=20)

The "technical" model seems to always select the `yellow` and `off` colors as the best images. The `yellow` ones aren't really what we want, because it misrepresents the color of pills. `Off` is also obviously not what we want, because you can't see anything.

It also chooses the way over exposed ones with `exposure = 2000` and `led-brightness = [1, 0.75]`. 

So we remove those below.

In [ ]:
crit = join_and([~has_yellow, ~is_off, ~(brightness_eq_1 & exposure2000), ~(brightness_eq_75 & exposure2000), ~(brightness_eq_1 & full_strip)])
cleaned = df[crit]
cleaned.nlargest(20, "score")

In [ ]:
open_images_matching_criteria(cleaned, count=20)

In [ ]:
display_images_matching_criteria(df, criteria=exposure2000)

So basically all the images with `exposure = 2000` are overexposed except when only the right strip is illuminated. Lets choose only those and take a look.

In [ ]:
display_images_matching_criteria(df, criteria=join_and([exposure2000, df["orientation"].str.contains("right-strip")]))

Almost all of those look pretty good so we'll keep them. Let's check out ones with `exposure = 1000`.

In [ ]:
display_images_matching_criteria(df, criteria=join_and([exposure1000, brightness_eq_1]))

So pictures with `exposure = 1000` and `brightnesss = 1` are all overexposed except when only right strip is illuminated.

# Aesthetic Model

In [ ]:
scores = load_data('output-aesthetic.txt')

In [ ]:
df = pandas.DataFrame(scores)
df

In [ ]:
df.describe()

In [ ]:
df.nlargest(10, "score")

In [ ]:
base = os.path.join("src", "")
base = "./src/tests/test_images/"
for idx, row in df.nlargest(20, "score").iterrows():
    print(idx)
    print(row["image_id"])
    img_file = os.path.join(base, row["image_id"] + ".jpg")
    img = Image(img_file)
    display(img)

In [ ]:
yellow = df["image_id"].str.contains("yellow") 
is_off = df["color"]=="off"
brightness_eq_1 = df["led-brightness"]==1
brightness_eq_75 = df["led-brightness"]==0.75
exposure1000 = df["exposure"]==1000
exposure2000 = df["exposure"]==2000
full_strip = df["orientation"]=="full-strip"

In [ ]:
cleaned = df[(~has_yellow) & (~is_off) & ~(brightness_eq_1 & exposure2000) & ~(brightness_eq_75 & exposure2000) & ~(brightness_eq_1 & exposure1000)]
cleaned

In [ ]:
cleaned.nlargest(20, "score")

In [ ]:
for idx, row in cleaned.nlargest(20, "score").iterrows():
    print(idx)
    print(row["image_id"])
    img_file = os.path.join(base, row["image_id"] + ".jpg")
    img = Image(img_file)
    display(img)

In [ ]:
not_exposure2000 = df[~has_exposure2000]
not_exposure2000

In [ ]:
not_exposure2000.nlargest(10, "score")

In [ ]:
for idx, row in not_exposure2000.nlargest(20, "score").iterrows():
    print(idx)
    print(row["image_id"])
    img_file = os.path.join(base, row["image_id"] + ".jpg")
    img = Image(img_file)
    display(img)

In [ ]:
not_yellow.nlargest(10, "score")

In [ ]:
for idx, row in not_yellow.nlargest(20, "score").iterrows():
    print(idx)
    print(row["image_id"])
    img_file = os.path.join(base, row["image_id"] + ".jpg")
    img = Image(img_file)
    display(img)